In [ ]:
import json
import re
import subprocess

from multiprocessing import JoinableQueue as Queue
from os import listdir, path

CAMERA_DB_PATH = "./metadata/cameras.json"

INPUT_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/FULL-0801"
OUTPUT_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/0801-500"

MV_CMD = 'mv "INPUT_PATH" "OUTPUT_PATH"'

In [ ]:
with open(CAMERA_DB_PATH, "r") as f:
  camera_data = json.load(f)

In [ ]:
def clean_name(inp):
  out_name = inp.replace(" ", "-").replace("Ã", "A")
  out_name = out_name.replace("(", "-").replace(")", "-")
  out_name = out_name.replace("-.", ".")
  out_name = re.sub(r'[-]+', '-', out_name)
  out_name = re.sub(r'[_]+', '_', out_name)
  out_name = re.sub(r'^([234])-ANDAR', r'\1o-ANDAR', out_name)
  out_name = re.sub(r'_08012023([0-9]{6})', r'_08012023_\1', out_name)
  return out_name

In [ ]:
to_process = Queue()

for input_dir in camera_data.keys():
  input_dir_path = path.join(INPUT_PATH, input_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])
  for input_file in input_files:
    output_file = clean_name(input_file)
    output_file_path = path.join(input_dir_path, output_file)
    if not path.exists(output_file_path):
      input_file_path = path.join(input_dir_path, input_file)
      to_process.put((input_file_path, output_file_path))

In [ ]:
while not to_process.empty():
    inp, outp = to_process.get()
    my_cmd = MV_CMD.replace("INPUT_PATH", inp).replace("OUTPUT_PATH", outp)
    print(my_cmd)
    ret = subprocess.run(my_cmd, shell=True)